## Importing libraries

In [1]:
#import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import re
#import time

import dask.dataframe as dd
#import dask.array as da
import dask.bag as db
import dask
from dask.distributed import Client, SSHCluster
import dask.distributed

# import hvplot.dask

## Cluster up

In [2]:
cluster = SSHCluster(
            ["bhbh-1", "bhbh-1", "bhbh-2", "bhbh-3", "bhbh-4", "bhbh-5"],
            connect_options={"client_keys": "/home/ubuntu/private/tbertola_key.pem"},
            worker_options={"n_workers": 4,
                            "nthreads": 4}, # because each bhbh-* has 4 cores
            scheduler_options={"port": 8786, "dashboard_address": ":8787"}
            )
client=Client(cluster)

2023-06-12 14:01:13,635 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:13,633 - distributed.scheduler - INFO - State start
2023-06-12 14:01:13,644 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:13,642 - distributed.scheduler - INFO -   Scheduler at:   tcp://10.67.22.140:8786
2023-06-12 14:01:14,619 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:14,617 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:33697'
2023-06-12 14:01:14,672 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:14,671 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:38847'
2023-06-12 14:01:14,675 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:14,675 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:39033'
2023-06-12 14:01:14,679 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:14,679 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:39051'
2023-06-12 14:01:15,495 - distributed.deploy.ssh - INFO - 2023-06-12

2023-06-12 14:01:17,189 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:17,182 - distributed.worker - INFO -       Start worker at:    tcp://10.67.22.31:39387
2023-06-12 14:01:17,191 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:17,182 - distributed.worker - INFO -          Listening to:    tcp://10.67.22.31:39387
2023-06-12 14:01:17,194 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:17,182 - distributed.worker - INFO -          dashboard at:          10.67.22.31:33623
2023-06-12 14:01:17,196 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:17,182 - distributed.worker - INFO - Waiting to connect to:    tcp://10.67.22.140:8786
2023-06-12 14:01:17,197 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:17,182 - distributed.worker - INFO - -------------------------------------------------
2023-06-12 14:01:17,198 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:17,183 - distributed.worker - INFO -               Threads:                          4
2023-06-12 14:01:17,200 - di

2023-06-12 14:01:18,107 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:18,104 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.220:35599'
2023-06-12 14:01:18,116 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:18,114 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.220:36313'
2023-06-12 14:01:18,118 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:18,116 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.220:33979'
2023-06-12 14:01:18,121 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:18,119 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.220:36033'
2023-06-12 14:01:18,248 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:18,244 - distributed.worker - INFO -       Start worker at:    tcp://10.67.22.21:37929
2023-06-12 14:01:18,249 - distributed.deploy.ssh - INFO - 2023-06-12 14:01:18,244 - distributed.worker - INFO -          Listening to:    tcp://10.67.22.21:37929
2023-06-12 14:01:18,250 - di

## Client 

In [3]:
client

<Client: 'tcp://10.67.22.140:8786' processes=20 threads=80, memory=155.26 GiB>

## Creating list of directories

## Creating Filename

In [32]:
filename_list = ['/mnt/bhbh/fiducial_Hrad_5M/sevn_output_Z0.0004A1L1/0/output_0.csv','/mnt/bhbh/fiducial_Hrad_5M/sevn_output_Z0.0004A1L1/0/evolved_0.dat', '/mnt/bhbh/fiducial_Hrad_5M/sevn_output_Z0.0004A1L1/0/logfile_0.dat']

## Creating a bag

In [33]:
bag = db.from_sequence(filename_list, npartitions=1)#.map(pd.read_table)

## Watching inside the bag

In [34]:
examples = bag.take(3)

print(examples)

('/mnt/bhbh/fiducial_Hrad_5M/sevn_output_Z0.0004A1L1/0/output_0.csv', '/mnt/bhbh/fiducial_Hrad_5M/sevn_output_Z0.0004A1L1/0/evolved_0.dat', '/mnt/bhbh/fiducial_Hrad_5M/sevn_output_Z0.0004A1L1/0/logfile_0.dat')


## Applying pre-processing function to the bag

In [37]:
output_column_to_remove = ['ID', 'MHE_0', 'MCO_0', 'Radius_0', 'RHE_0','RCO_0', 'Luminosity_0', 'Temperature_0', 'Lambda_0',
                               'Phase_0', 'PhaseBSE_0', 'Zams_0', 'MHE_1', 'MCO_1','Radius_1', 'RHE_1', 'RCO_1', 
                               'Luminosity_1', 'Temperature_1','Lambda_1', 'Phase_1', 'PhaseBSE_1', 'Zams_1']

output_column_to_read = ['name', 'Mass_0', 'RemnantType_0', 'Mass_1', 'RemnantType_1',
                         'Semimajor','Eccentricity','GWtime','BWorldtime']

output_column_type = ['string', 'float64', 'int64', 'float64', 'int64',
                      'float64', 'float64', 'float64', 'float64', 'int64']

evolved_column_to_read = ['name', 'Mass_0', 'Z_0', 'SN_0', 'Mass_1', 'SN_1', 'a', 'e']


evolved_column_type = ['string', 'float64', 'float64', 'string', 'float64', 
                      'string', 'float64', 'float64']

#evolved_column_to_remove = ['#ID', 'spin_0', 'Tstart_0', 'spin_1', 'Tstart_1', 'Tend', 'Dtout', 'Seed']
drop_list = ['RemnantType_0',  'RemnantType_1']

In [40]:
def preprocessing_bag_of_thread(bag_of_thread, output_column_to_read, output_column_type, evolved_column_to_read, evolved_column_type, drop_list):
    
    paths = bag_of_thread.take(3)
    
   
    #preprocessing output
    
    output = pd.read_csv(paths[0], usecols=output_column_to_read, dtype=dict(zip(output_column_to_read, output_column_type))).\
                rename(columns={'Mass_0':'Mass_0_out', 'Mass_1':'Mass_1_out'})

    idxBHBH=(output.RemnantType_0==6) & (output.RemnantType_1==6) & (output.Semimajor.notnull())
    output=output[idxBHBH]
    
    df=output
    
    
    #preprocessing evolved
    
    alpha = float(re.findall(r".+(?<=A)(.*)(?=L)", paths[1])[0])
    
    evolved = pd.read_table(paths[1], sep='\s+', usecols=evolved_column_to_read, dtype=dict(zip(evolved_column_to_read, evolved_column_type)))                
    
    evolved['alpha'] = alpha
    
    #preprocessing logfile
    
    logfile = pd.read_csv(paths[2], header=None)

    df_RLO = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);RLO_BEGIN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='RLO').\
                reset_index()

    df_CE = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);CE;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='CE').\
                reset_index()

    df_BSN = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);BSN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='BSN').\
                reset_index()

    
    #merge
    bhbh = evolved.merge(output, on=['name'], how='inner').\
                   merge(df_RLO, on=['name'], how='left').\
                   merge(df_CE,  on=['name'], how='left').\
                   merge(df_BSN, on=['name'], how='left').\
                   fillna(value=0).\
                   drop(columns=drop_list)
    
    #add some useful columns
    bhbh['tdelay'] = bhbh['GWtime'] + bhbh['BWorldtime']

    bhbh['Mass_max_out'] = bhbh['Mass_1_out']
    bhbh['Mass_max_out'] = bhbh['Mass_max_out'].\
                            where(cond=(bhbh['Mass_max_out'] > bhbh['Mass_0_out']), other=bhbh['Mass_0_out'])

    bhbh['q'] = bhbh['Mass_1_out']/bhbh['Mass_0_out']
    bhbh['q'] = bhbh['q'].\
                where(cond=(bhbh['Mass_1_out'] < bhbh['Mass_0_out']), other=bhbh['Mass_0_out']/bhbh['Mass_1_out'])

    bhbh['Mass_chirp'] = ((bhbh['Mass_0_out'] * bhbh['Mass_1_out'])**(3/5))/((bhbh['Mass_0_out'] + bhbh['Mass_1_out'])**(1/5))
    
    return bhbh

In [44]:
%%time
df = preprocessing_bag_of_thread(bag, output_column_to_read, output_column_type, evolved_column_to_read, evolved_column_type, drop_list)

CPU times: user 17.9 s, sys: 3 s, total: 20.9 s
Wall time: 21 s


In [42]:
df.head()

,name,Mass_0,Z_0,SN_0,Mass_1,SN_1,a,e,alpha,Mass_0_out,...,Eccentricity,GWtime,BWorldtime,RLO,CE,BSN,tdelay,Mass_max_out,q,Mass_chirp
0,0_186500805616303,24.025,0.0004,rapid_gauNS,15.586,rapid_gauNS,56.1,0.00704,1.0,9.050055,...,0.028653,7.108607e+03,10.624610,2.0,0.0,2,7.119232e+03,9.050055,0.928217,7.589440
1,0_502130275753280,39.240,0.0004,rapid_gauNS,37.312,rapid_gauNS,1180.0,0.07820,1.0,38.336310,...,0.006686,1.274892e+09,5.217448,2.0,0.0,2,1.274892e+09,38.336310,0.950738,32.539204
2,0_201673565337120,61.947,0.0004,rapid_gauNS,30.179,rapid_gauNS,5040.0,0.22100,1.0,59.190780,...,0.277687,1.390318e+12,6.160173,0.0,0.0,2,1.390318e+12,59.190780,0.357987,30.060437
3,0_929528790266714,135.386,0.0004,rapid_gauNS,78.408,rapid_gauNS,20600.0,0.75400,1.0,47.067630,...,0.777802,4.464742e+14,3.399872,0.0,0.0,2,4.464742e+14,47.067630,0.742702,35.234364
4,0_583722007414750,51.010,0.0004,rapid_gauNS,50.206,rapid_gauNS,2400.0,0.29900,1.0,49.445770,...,0.005092,3.798065e+09,4.275696,2.0,0.0,2,3.798065e+09,49.445770,0.985839,42.738950


In [4]:
## bag_preprocessed = bag.map(preprocessing)

NameError: name 'bag' is not defined

## Converting to a bag of dataframe